### Generate Cambdrige Data

This script leverages the CBECI API to download fresh Bitcoin energy consumption estimates from the [Cambridge Centre for Alternative Finance](https://ccaf.io/cbeci/index) and output the best guess annualized data for all energy price assumptions into a single CSV.

The motivation for this script is that the Cambridge Bitcoin Electricity Consumption Index assumes a constant fixed energy cost of $0.05/kWh, but in the face of rising energy prices there is desire to modify the output of the index to be more reflective of dynamic energy prices.

The output of this script is designed to be paired with EIA energy price data to allow for an evolving adjustment of what level of electricity spend the Bitcoin Network can support.

- Author: [Data Always](dataalways.substack.com)
- Last Modified: November 15, 2022

In [1]:
import numpy as np
import pandas as pd

In [2]:
"""
The analysis in this repository is only concerned with the CBECI
`BEST GUESS`. To avoid polution of the .csv we drop all the 
other columns. 
"""
def drop_standard_columns(dataframe):
    return dataframe.drop([
        'Timestamp', 
        'power MAX, GW', 
        'power MIN, GW', 
        'power GUESS, GW', 
        'annualised consumption MAX, TWh',
        'annualised consumption MIN, TWh'
        ], 
        axis= 1
    )

In [3]:
# Create the base dataframe on which to merge others by looking at the $0.01 data
df = pd.read_csv(
    "https://ccaf.io/cbeci/api/v1.2.0/download/data?price=0.01", 
    skiprows=1
)

# Drop the unwanted columns
df = drop_standard_columns(df)

# Rename the columns
df.rename(
    columns={
        'Date and Time': 'date', 
        'annualised consumption GUESS, TWh': 0.01
    }, 
    inplace=True
)

In [4]:
# Download the data for all prices and then merge them into one dataframe
for elec_cost in np.linspace(0.02, 0.19, 18):
    elec_str = "{:.2f}".format(elec_cost)
    
    tmp_df = pd.read_csv(
        f"https://ccaf.io/cbeci/api/v1.2.0/download/data?price={elec_str}", 
        skiprows=1
    )
    
    tmp_df = drop_standard_columns(tmp_df)
    
    tmp_df.rename(
        columns={
            'Date and Time': 'date', 
            'annualised consumption GUESS, TWh': elec_str
        }, 
        inplace=True
    )

    # Merge the data into the main data frame
    df = df.merge(
        tmp_df, 
        how="inner", 
        on="date"
    )

In [5]:
df.tail()

,date,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.10,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19
4506,2022-11-18T00:00:00,160.596392,138.218651,115.948885,108.276794,96.320211,87.315394,79.899780,72.105232,68.337947,59.990973,54.177153,54.177153,63.26991,52.113412,57.618272,50.858529,48.843833,45.716266,42.366663
4507,2022-11-19T00:00:00,156.014018,134.800356,112.806032,105.418635,93.363672,84.845931,77.036192,70.234784,65.722291,57.437095,53.094668,52.661704,63.26991,52.113412,57.618272,50.858529,48.843833,45.716266,42.366663
4508,2022-11-20T00:00:00,157.841282,137.003098,114.753982,106.655603,94.178746,85.655675,77.406295,71.314045,66.049821,57.386847,54.181004,53.202946,63.26991,52.113412,57.618272,50.858529,48.843833,45.716266,42.366663
4509,2022-11-21T00:00:00,157.698406,137.402192,115.202755,106.559034,93.966689,85.520358,77.101828,71.495547,65.284735,57.050694,54.541526,54.068934,63.26991,52.113412,57.618272,50.858529,48.843833,45.716266,42.366663
4510,2022-11-22T00:00:00,151.965909,133.536091,111.619524,102.679053,90.602013,82.827047,74.007951,69.250940,62.123495,54.638127,53.006885,53.202976,63.26991,52.113412,57.618272,50.858529,48.843833,45.716266,42.366663


In [6]:
# Export the data to a .csv file
df.to_csv(
    './datasets/cbeci-annualized-consumption.csv', 
    index = False
)